### * While the original source code is written in Pytorch, the below code is adapted to Tensorflow.

- GPU utilization not enabled

# 1. Preparing the tinyshakespeare text file for training

In [23]:
# Downloading tinyshakesphere for training
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > tinyshakespeare.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  5208k      0 --:--:-- --:--:-- --:--:-- 5211k


In [24]:
# Inspecting the text file
with open('tinyshakespeare.txt','r') as file:
    text = file.read()
print(f'There are {len(text)} characters in the dataset')

There are 1115394 characters in the dataset


In [25]:
# Printing the first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [26]:
# Identifying the number of unique characters contained in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Number of unique characters (including white space): {vocab_size}{''.join(chars)}")

Number of unique characters (including white space): 65
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


# 2. Basic mapping between characters to integers

Tokenizing at the character-level.

More sophisticated examples of word encoding include Google's SentencePiece and OpenAI's tiktoken

In [27]:
# Assigning numbers to each characters to encode the characters to integers
ctoi = {char : num for num, char in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
print(encode('Shakespeare in digits'))

# Reversely, decode integers back to characters
itoc = {num : char for num, char in enumerate(chars)}
decode = lambda l : ''.join([itoc[i] for i in l])
print(decode(encode('Shakespeare in digits')))

[31, 46, 39, 49, 43, 57, 54, 43, 39, 56, 43, 1, 47, 52, 1, 42, 47, 45, 47, 58, 57]
Shakespeare in digits


In [28]:
# Tokenizing the total text
import tensorflow as tf
data = tf.convert_to_tensor(encode(text))
print(data.shape, data.dtype)
print(data[:100])

(1115394,) <dtype: 'int32'>
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int32)


In [29]:
# Train and validation split sets, with 9:1 ratio
n = int(0.9*len(data))
data_train = data[:n]
data_test = data[n:]
print(f'Length of train data : {len(data_train)}\nLength of test data : {len(data_test)}')

Length of train data : 1003854
Length of test data : 111540


In [30]:
# Starting with block_size implementation
block_size = 8                            # Context length
print(data_train[:block_size + 1])
x = data_train[:block_size]               # Initial block-size
y = data_train[1:block_size+1]            # Next block-size
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Input : {context}, Output : {target}')

tf.Tensor([18 47 56 57 58  1 15 47 58], shape=(9,), dtype=int32)
Input : [18], Output : 47
Input : [18 47], Output : 56
Input : [18 47 56], Output : 57
Input : [18 47 56 57], Output : 58
Input : [18 47 56 57 58], Output : 1
Input : [18 47 56 57 58  1], Output : 15
Input : [18 47 56 57 58  1 15], Output : 47
Input : [18 47 56 57 58  1 15 47], Output : 58


In [31]:
## To be worked on : packaging the code with script with variables for later
# Depiction of the chunk(or in here, block)-wise transformation.
# Having varied blocksize allows the algorithm to take into account the context for inference purpose

tf.random.set_seed(1337) # For reproducibility, to be sure to have consistent random number
batch_size = 4 # The number of independent sequences to train in parallel
block_size = 8 # The maximum context length for prediction

def get_batch(split):
    '''
    Function to generate a small batch of data of inputs x and targets y
    '''

    data = data_train if split == 'train' else data_test
    # Retrieving batches randomly
    ix = tf.random.uniform(shape = (batch_size,),
                          maxval = len(data) - block_size,
                          dtype = tf.int32)
    # Stacking the list of tensors
    x = tf.stack([data[i:i+block_size] for i in ix])
    y = tf.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
for batch in range(batch_size):       # Batch dimension
    for block in range(block_size):   # Time dimension
        context = xb[batch, :block+1]
        target = yb[batch, block]
        print(f'When input is {context.numpy().tolist()} the target is {target}')

inputs:
(4, 8)
tf.Tensor(
[[ 1 51 63  1 41 53 39 58]
 [39 42  0 20 47 57  1 52]
 [32 53  1 56 43 60 43 50]
 [54 39 52 63  1 54 47 43]], shape=(4, 8), dtype=int32)
targets:
(4, 8)
tf.Tensor(
[[51 63  1 41 53 39 58  6]
 [42  0 20 47 57  1 52 39]
 [53  1 56 43 60 43 50  1]
 [39 52 63  1 54 47 43 41]], shape=(4, 8), dtype=int32)
When input is [1] the target is 51
When input is [1, 51] the target is 63
When input is [1, 51, 63] the target is 1
When input is [1, 51, 63, 1] the target is 41
When input is [1, 51, 63, 1, 41] the target is 53
When input is [1, 51, 63, 1, 41, 53] the target is 39
When input is [1, 51, 63, 1, 41, 53, 39] the target is 58
When input is [1, 51, 63, 1, 41, 53, 39, 58] the target is 6
When input is [39] the target is 42
When input is [39, 42] the target is 0
When input is [39, 42, 0] the target is 20
When input is [39, 42, 0, 20] the target is 47
When input is [39, 42, 0, 20, 47] the target is 57
When input is [39, 42, 0, 20, 47, 57] the target is 1
When input is [39,

## Basic BigramModel for training

In [32]:
tf.random.set_seed(1337)

class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, vocab_size)

    def call(self, idx, targets=None):
        '''Method for loss calculation, based on idx (input token indices) and
        target (target token indices)
        B : Batch size
        T : Time = block size = sequence length
        C : Channel = vocab size = number of classes
        '''
        logits = self.token_embedding_table(idx)  # Replacing embedding to the indices

        if targets is None: # If target is not provided
            loss = None
        else:               # If target is provided, reshape the tensor so that it's compatible with categorical cross entropy
            B, T, C = tf.shape(logits) # Get the shape of logits
            logits = tf.reshape(logits, (B * T, C)) # Flatten logits for comparison
            targets = tf.reshape(targets, (B * T,)) # Flatten targets
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        Text generating method
        '''
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self(idx)
            # Focus only on the last time step (i.e. history is not being used)
            logits = logits[:, -1, :]  # becomes (B, C)
            # Apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # One sample prediction from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64) # (B, 1)

            # idx_next = tf.random.categorical(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, tf.cast(idx_next, tf.int32)], axis=1)  # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)

logits, loss = model.call(xb, yb)
print(logits.shape)
print(loss.numpy())

print(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))

(32, 65)
4.185914

sZTe-Wz-K,?hNk?Yr:r'KUFLHH:QmLbpCkI
oYwnqePrE
!zgz'U:,?ZgzxEjHtgpzBQjGjM&vv.;OBdqFlQ qxcwcexWhPKs:$'


### Creating an optimizer

In [47]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate = 1e-3) ## Learning rate of 0.001

In [34]:
eval_iters = 400

# @tf.function omitted as iterating over a symbolic `tf.Tensor` is not allowed.
def estimate_loss():
  '''
  Function to average up the loss in multiple batches for both splits
  '''
  output = {}
  model.trainable = False # Setting the model to evaluation phase
  for split in ['train','val']:
      losses = []
      for _ in range(eval_iters):
          X, Y = get_batch(split)
          logits, loss = model.call(X,Y)
          losses.append(loss)
      output[split] = tf.reduce_mean(losses)
  model.trainable = True # Setting the model back to training phase
  return output

In [35]:
batch_size = 32
for step in tf.range(4000):
    if step % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss and update parameters
    with tf.GradientTape() as tape:
        logits, loss = model(xb,yb)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print(f'Final Loss: {loss.numpy()}')

Step 0: train loss 4.1761, val loss 4.1760
Step 400: train loss 3.4000, val loss 3.4083
Step 800: train loss 3.0095, val loss 3.0167
Step 1200: train loss 2.8002, val loss 2.8070
Step 1600: train loss 2.6759, val loss 2.6935
Step 2000: train loss 2.6103, val loss 2.6269
Step 2400: train loss 2.5720, val loss 2.5824
Step 2800: train loss 2.5461, val loss 2.5476
Step 3200: train loss 2.5226, val loss 2.5310
Step 3600: train loss 2.5044, val loss 2.5202
Final Loss: 2.5085837841033936


In [36]:
# Generate a sequence
idx = tf.zeros((1, 1), dtype=tf.int32)
generated_sequence = model.generate(idx, max_new_tokens=1000).numpy()
print(decode(generated_sequence[0].tolist()))



Fathe wivor thiranen as hy psce lf tes: tangeisnershert ma forerdve wilite
GHoprs be uelathe?xou Ingrinme ll llou ond ahit Erendir
D bloouHisu; sh he l la!&ily set: fueyoeke fo t MPris;
FR.
Pres;
Wollouth
I n vesere his.

I dg
Weronth tournd woour ariets iend t I bist p'CKy: s d$XTolfes titowil, irchatigherou
Bun
KExfor ife oth, futcy

Th d mowethest htover h ht wll anl XHaveirpril aik r ver?

The
D: tn
HAUSthe. e Pl ony stan fof, b, t w,
TYHUNy mit amad
DYI whaf she she.
SAPEO:
CKIOMOx:
CFas t t ace ben?
YUMaco thapoush, tes flyofen pea slare is.
K:TIf t:
may s, pend!K?
AGNDII s ist theaxcelevers; therof nt; tey, d t hyClschathice te oRD:
PO as ipq!
Nin, m n n



We,
NED&e h.
Aneroorot y rcldd icucRORDEur YK:
D t um.
SLO d aASur;

WEmeaiso'?
P?Vrey th sel'ty tind gu figardstarthecore and withes ousthad besthind w st s wrmomy, toxthe
Dzhe apon
DI mbe, bilorey f ws he wurane mopoy urswicaze mprs se in taiuleayM:



ME.
QYo

HAn hedwed sthadr the ind, tou fo erthicacoforil worow thavea

## The mathematical trick in self-attention
Below present different ways of calculating weighted aggregation of a matrix, from beginning of the block in each batch, up to the 't'th token. The results of the four approaches are the same

In [37]:
# Common code
tf.random.set_seed(1337)

# Tokens learning from previous context, by calculating average up to 't'th token
B,T,C = 4,8,2 # Batch, Time, Channels
x = tf.random.uniform(shape=(B, T,C))

### Version 1: Basic

In [38]:
xbow = tf.zeros((B,T,C)) # Defining a bag of words
for b in range (B):
    for t in range (T):
        xprev = x[b, :t+1] # (t, C) Batch, including the 't'th token
        xbow = xbow.numpy()  # Convert xbow to numpy array to support assignment
        xbow[b, t] = tf.reduce_mean(xprev, axis=0).numpy()  # Calculate mean and assign to xbow
        xbow = tf.convert_to_tensor(xbow)  # Convert back to tensor

### Version 2: Vectorizing


In [39]:
w = tf.linalg.band_part(tf.ones((T,T)),num_lower = 8, num_upper= 0)
w = w / tf.math.reduce_sum(w, axis = 1, keepdims = True) # Low triangular matrix for calculating average weights

xbow2 = w @ x # (B, T, T) @ (B , T, C) --> (B, T, C)
tf.experimental.numpy.allclose(xbow,xbow2).numpy() # Checking whether xbow == xbow2

True

### Version 3: Using softmax

In [40]:
tril = tf.linalg.band_part(tf.ones((T,T)),num_lower = 8, num_upper= 0)
w = tf.zeros((T,T))
w = tf.where(tril == 0, float('-inf'), w) # Replacing 0s with -inf, indicating that the past blocks cannot communicate with the future blocks
w = tf.nn.softmax(w, axis = -1) # Normalizing the weight matrix
xbow3 = w @ x
tf.experimental.numpy.allclose(xbow,xbow3).numpy()

True

### Version 4: Self-attention

Called self-attention as the key, query and value are generated from the same value (x)

Note that key and query weights values are different as

In [41]:
# Attention mechanism
head_size = 16
key = tf.keras.layers.Dense(units=head_size, use_bias=False)
query = tf.keras.layers.Dense(units=head_size, use_bias=False)
value = tf.keras.layers.Dense(units=head_size, use_bias=False)
k = key(x) # Weights adjusted, (B, T, 16)
q = query(x) # (B, T, 16)
w = q @ tf.transpose(k, perm=[0,2,1]) # (B, T, 16) @ (B, 16, T) -> (B, T, T), with (T, T) indicating elements compared with every element in the sequence

tril = tf.linalg.band_part(tf.ones((T,T)),num_lower = 8, num_upper= 0)
w = tf.where(tril == 0, float('-inf'), w) # Replacing 0s with -inf, indicating that the past blocks cannot communicate with the future blocks
w = tf.nn.softmax(w, axis = -1) # Normalizing the weight matrix

v = value(x)
out = w @ v # Using aggregated value instead of the raw x for dimensionality reduction, information extraction
out.shape

TensorShape([4, 8, 16])

Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over as a set of vectors. This is why we need to positionally encode tokens
- Each example across batch dimension is of course processed completely independently and never 'talk' to each other
- In an 'encoder' attention block (w = tf.where(tril == 0, float('-inf'), w))code can be omitted, allowing all tokens to communicate. This block here is called a 'decoder' attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- 'Self attention' just means that the keys and values are produced from the same source as queries. In 'Cross-attention', the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- 'Scaled' attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, wei will be unit variance too and softmax will stay diffuses and not saturate too much, Illustration below

## Modified BigramModel with self-attention

In [42]:
# Hyperparameters
batch_size = 32 # Independent sequences to process in parallel
block_size = 8 # Maximum context length for prediction
max_iters = 5000
eval_interval = 50
learning_rate = 1e-3
eval_iters = 200
n_embed = 32

In [45]:
tf.random.set_seed(1337)

class Head(tf.keras.Model):
    """one head of self-attention"""

    def __init__(self, head_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(Head, self).__init__()
        self.key = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.tril = tf.constant(tf.linalg.band_part(tf.ones((block_size, block_size)), -1, 0), dtype= tf.float32)

    def call(self, x):
        B, T, C = x.shape
        k = self.key(x)     # (B,T,C)
        q = self.query(x)   # (B,T,C)
        # Compute attention scores ('affinities')
        wei = q @ tf.transpose(k, perm=[0,2,1]) * C ** (-0.5) # (B,T,C) @ (B,C,T) -> (B,T,T)
        wei = tf.where(self.tril[:T, :T] == 0, float('-inf'), wei) # Mask the upper triangular part, (B,T,T)
        wei = tf.nn.softmax(wei, axis = -1) # (B,T,T)
        # Perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out



class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.position_embedding_table = tf.keras.layers.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed)
        self.lm_head = tf.keras.layers.Dense(units=vocab_size)

    def call(self, idx, targets=None):
        '''Method for loss calculation, based on idx (input token indices) and
        target (target token indices)
        B : Batch size
        T : Time = block size = sequence length
        C : Channel = vocab size = number of classes
        '''
        B,T = idx.shape

        token_emb = self.token_embedding_table(idx)  # (B, T, C) Replacing indices with embeddings
        pos_emb = self.position_embedding_table(tf.range(T, dtype=tf.int32)) # (T,C)
        x = token_emb + pos_emb # (B, T, C) Containing both token embedding and position
        x = self.sa_head(x) # Apply one head of self-attention (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)

        if targets is None: # If target is not provided
            loss = None
        else:               # If target is provided, reshape the tensor so that it's compatible with categorical cross entropy
            B, T, C = tf.shape(logits) # Get the shape of logits
            logits = tf.reshape(logits, (B * T, C)) # Flatten logits for comparison
            targets = tf.reshape(targets, (B * T,)) # Flatten targets
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        Text generating method
        '''
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Crop idx to the last block_size tokens to avoid going out of scope
            idx_cond = idx[:, -block_size:]
            # Get the predictions
            logits, loss = self(idx_cond)
            # Focus only on the last time step (i.e. history is not being used)
            logits = logits[:, -1, :]  # becomes (B, C)
            # Apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # One sample prediction from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64) # (B, 1)

            # idx_next = tf.random.categorical(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, tf.cast(idx_next, tf.int32)], axis=1)  # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)

logits, loss = model.call(xb, yb)
print(logits.shape)
print(loss.numpy())

print(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))

(256, 65)
4.173274

saTf-Wz-K-?hNk;Yr:r'KUFLHH:QmLbpClI
oYwnqePrE
!zgz'U:,?ZgzxEjHtgpzAQjGjM&vv.;OBdqFlP qxcwcexWhPKs:$'


In [48]:
for step in tf.range(max_iters):
    if step % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss and update parameters
    with tf.GradientTape() as tape:
        logits, loss = model(xb,yb)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print(f'Final Loss: {loss.numpy()}')

Step 0: train loss 4.1738, val loss 4.1741
Step 200: train loss 3.0578, val loss 3.0937
Step 400: train loss 2.7313, val loss 2.7398
Step 600: train loss 2.6229, val loss 2.6082
Step 800: train loss 2.5584, val loss 2.5564
Step 1000: train loss 2.5136, val loss 2.5261
Step 1200: train loss 2.4928, val loss 2.4893
Step 1400: train loss 2.4776, val loss 2.4862
Step 1600: train loss 2.4662, val loss 2.4733
Step 1800: train loss 2.4550, val loss 2.4414
Step 2000: train loss 2.4447, val loss 2.4607
Step 2200: train loss 2.4414, val loss 2.4354
Step 2400: train loss 2.4224, val loss 2.4276
Step 2600: train loss 2.4263, val loss 2.4249
Step 2800: train loss 2.4199, val loss 2.4241
Step 3000: train loss 2.4157, val loss 2.4192
Step 3200: train loss 2.4099, val loss 2.4100
Step 3400: train loss 2.4038, val loss 2.4105
Step 3600: train loss 2.4016, val loss 2.4223
Step 3800: train loss 2.3983, val loss 2.4150
Step 4000: train loss 2.3998, val loss 2.4108
Step 4200: train loss 2.4044, val loss 2.

In [49]:
# Generate a sequence
idx = tf.zeros((1, 1), dtype=tf.int32)
generated_sequence = model.generate(idx, max_new_tokens=1000).numpy()
print(decode(generated_sequence[0].tolist()))


LARIHA:
Tovourat I hat arg imenf irmak, awbe, tiar best head fains sit ite sy bechepr'd as tracod tse fo gee,
Goru be?
Cand my mbrat, cheest had xby.

PEESNort irdo ssknghathereay if'l yorm ss angtrt Arosan ond hese wet. Hoverkud, me wisow man sth thabyowingoindn whimor thy ined at ms he arat cillo.
Thish by
Bajureese stheer andisuseso!
Hont, bre ay tae is here b howrard umy.

TOFLUET:
us hadele hanmomere gtthe; Alld cor blich, ay.

NELV:

Theastoh ad myomde hearliscit ghe! maf Senat of yummy ry wor ingey maverthe: Joutrllowle, TRHorpemonoongwono yeamatishe, ped's;
ARe INource!
PORIDWhirdyo fito gwhrer harimeay ho for dop!

ONors ilveve lorein nigg to wins! Bvostthew dy; irtatr is asimlece, cas?

OF; fo't burnge nt houcurt weve une.

AMURI: I fes
Bon
Hesist hond sus lay anolllay, oay Vetrmemcengo thin for lclo cll-us:
The paeeates
You ise wor witwinghe seagre ut a rt igu eakefary nout menod.
POLond, hid mher:
RY:
Bin? haw diveramrer senurf llld fer cavely cknet te thay the sond
D ingo

## Multi-headed attention

In [ ]:
tf.random.set_seed(1337)
n_embed = 32


class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.lm_head = tf.keras.layers.Dense(units=vocab_size)

    def call(self, idx, targets=None):
        '''Method for loss calculation, based on idx (input token indices) and
        target (target token indices)
        B : Batch size
        T : Time = sequence length = block size
        C : Channel = number of classes = vocab size
        '''
        token_emb = self.token_embedding_table(idx)  # Replacing embedding to the indices
        pos_emb = self.position_embedding_table(tf.range(T, dtype=tf.int32)) # (T,C)
        x = token_emb + pos_emb # (B,T,C)
        logits = self.lm_head(x  ) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshaping the tensor so that it's compatible with categorical cross entropy
            B, T, C = tf.shape(logits) # Get the shape of logits
            logits = tf.reshape(logits, (B * T, C)) # Flatten logits for comparison
            targets = tf.reshape(targets, (B * T,)) # Flatten targets
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        Text generating method
        '''
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample prediction from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64)

#            idx_next = tf.random.categorical(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, tf.cast(idx_next, tf.int32)], axis=1)  # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model.call(xb, yb)
print(logits.shape)
print(loss.numpy())

print(decode(model.generate(idx=tf.zeros((1, 1), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))


AttributeError: 'BigramLanguageModel' object has no attribute 'position_embedding_table'

In [ ]:
# Video from 1:00:00. Need to recheck how self-attention functions

# Version 4: self-attention
tf.random.set_seed(1337)
B,T,C = 4, 8, 32 # batch, time, channels
x = tf.random.normal(shape=(B, T, C))

# Single head perform self-attention
head_size = 16
key = tf.keras.layers.Dense(units=head_size, use_bias=False)
query = tf.keras.layers.Dense(units=head_size, use_bias=False)
value = tf.keras.layers.Dense(units=head_size, use_bias=False)

# All tokens in all positions produce independent key and query
k = key(x) # B, T, 16
q = query(x) # B, T, 16
# Communicating key with query
w = q @ tf.transpose(k, perm=[0,2,1]) # (B, T, 16) @ (B, 16, T) -> (B,T,T)

tril = tf.linalg.band_part(tf.ones((T,T)),num_lower = 8, num_upper= 0)
w = tf.zeros((T,T))
w = tf.where(tril == 0, float('-inf'), w) # Upper triangular masking, indicating future bow cannot communicate with the past
w = tf.nn.softmax(w, axis = -1) # normalizing the weight matrix

v = value(x)
out = w @ v
#out = w @ x # K : private information

out.shape

TensorShape([4, 8, 16])

In [ ]:
# Verifying whether the row sum of weights equal 1
import numpy as np
# Convert TensorFlow tensor to NumPy array
w_np = w.numpy()

# Set NumPy print options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the tensor
print(w_np.round(6))

[[1.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.5      0.5      0.       0.       0.       0.       0.       0.      ]
 [0.333333 0.333333 0.333333 0.       0.       0.       0.       0.      ]
 [0.25     0.25     0.25     0.25     0.       0.       0.       0.      ]
 [0.2      0.2      0.2      0.2      0.2      0.       0.       0.      ]
 [0.166667 0.166667 0.166667 0.166667 0.166667 0.166667 0.       0.      ]
 [0.142857 0.142857 0.142857 0.142857 0.142857 0.142857 0.142857 0.      ]
 [0.125    0.125    0.125    0.125    0.125    0.125    0.125    0.125   ]]


In [ ]:
q = tf.random.normal((B, T, head_size))
k = tf.random.normal((B, T, head_size))

# Calculate the weights
wei = q @ tf.transpose(k, perm=[0, 2, 1]) * (head_size ** -0.5)

In [ ]:
tf.math.reduce_variance(k).numpy()

0.9939072

In [ ]:
tf.math.reduce_variance(q).numpy()

0.9280848

In [ ]:
tf.math.reduce_variance(wei).numpy()

0.9270358

In [ ]:
x.shape

TensorShape([4, 8, 32])

In [ ]:
head_size = 6
max_iters = 2000
eval_interval = 500
learning_rate = 1e-2
eval_iters = 200
n_embed = 384
batch_size = 64
block_size = 256
dropout = 0.3

n_layer = 3

class Head(tf.keras.Model):
    """one head of self-attention"""

    def __init__(self, head_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(Head, self).__init__()
        self.key = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.tril = tf.constant(tf.linalg.band_part(tf.ones((block_size, block_size)), -1, 0), dtype= tf.float32)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x):
        B, T, C = x.shape
        k = self.key(x)     # (B,T,C)
        q = self.query(x)   # (B,T,C)
        # compute attention scores ('affinities')
        wei = tf.matmul(q, k, transpose_b=True) * (C ** -0.5) # (B,T,C) @ (B,C,T ) -> (B,T,T))
        wei = tf.where(self.tril[:T, :T] == 0, float('-inf'), wei) # Mask the upper triangular part
        wei = tf.nn.softmax(wei, axis = -1)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = tf.matmul(wei, v) # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out

class FeedForward(tf.keras.layers.Layer):
    '''A simple linear layer followed by a non-linearity'''
    def __init__(self, n_embed):
        super().__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Dense(4 * n_embed), # (n_embed, 4 * n_embed)
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(n_embed), # (4 * n_embed, n_embed)
            tf.keras.layers.Dropout(dropout),
        ])

    def call(self, x):
        return self.net(x)


class MultiHeadAttention(tf.keras.layers.Layer):
    '''Multiple heads of self-attention in parallel'''

    def __init__(self,num_heads, head_size):
        super().__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.projection = tf.keras.layers.Dense(n_embed)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x):
        out = tf.concat([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.projection(out))
        return out

class Block(tf.keras.layers.Layer):
    """Transformer blocks : communication followed by computation"""

    def __init__(self, n_embed, n_head):
        # n_embed : embedding dimension, n_head : the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = tf.keras.layers.LayerNormalization(axis=-1)
        self.ln2 = tf.keras.layers.LayerNormalization(axis=-1)

    def call(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(tf.keras.Model):

    def __init__(self, vocab_size):
        '''Initializing embedding layer, which maps integer indices to
        dense vectors of vocab size'''
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.position_embedding_table = tf.keras.layers.Embedding(block_size, n_embed)
        self.blocks = tf.keras.Sequential([Block(n_embed = n_embed, n_head = 4) for _ in range(n_layer)])
        self.ln_f = tf.keras.layers.LayerNormalization(axis=-1) # final layer normalization
        self.lm_head = tf.keras.layers.Dense(units=vocab_size)

    def call(self, idx, targets = None):
        '''Method for loss calculation, based on idx (input token indices) and
        target (target token indices)
        B : Batch size
        T : Time = sequence length = block size
        C : Channel = number of classes = vocab size
        '''
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        token_emb = self.token_embedding_table(idx) # (B,T,C)
        position_emb = self.position_embedding_table(tf.range(T, dtype=tf.int32)) # (T, C)
        x = token_emb + position_emb # (B,T,C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            # Reshaping the tensor so that it's compatible with categorical cross entropy
            B, T, C = tf.shape(logits) # Get the shape of logits
            logits = tf.reshape(logits, (B * T, C)) # Flatten logits for comparison
            targets = tf.reshape(targets, (B * T,)) # Flatten targets
            loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True))
        return logits, loss

    def generate(self, idx, max_new_tokens):
        '''
        Text generating method
        '''
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample prediction from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int32)
            # append sampled index to the running sequence
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model.call(xb, yb)
print(logits.shape)
print(loss.numpy())



optimizer = tf.keras.optimizers.Adam(learning_rate)

for step in tf.range(max_iters):
    if step % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    with tf.GradientTape() as tape:
        logits, loss = model(xb,yb)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

print(f'Final Loss: {loss.numpy()}')

print(decode(model.generate(idx=tf.zeros((1, block_size), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))


(16384, 65)
4.9176507
Step 0: train loss 4.9358, val loss 4.9593
Step 200: train loss 3.0522, val loss 3.0692
Step 400: train loss 2.7568, val loss 2.7682
Step 600: train loss 2.6052, val loss 2.6109
Step 800: train loss 2.5359, val loss 2.5471
Step 1000: train loss 2.5138, val loss 2.5287
Step 1200: train loss 2.4898, val loss 2.5081
Step 1400: train loss 2.5138, val loss 2.5282
Step 1600: train loss 2.5075, val loss 2.5315
Step 1800: train loss 2.4910, val loss 2.5144
Final Loss: 2.4965360164642334
































































































































































































































































ORO tharthy,
Ost veapy t ours,
Auladeane wese t
Ol thlent ug he s?
In,
ncordllder urild:
nouye aveet


In [ ]:
print(decode(model.generate(idx=tf.zeros((1, block_size), dtype=tf.int32), max_new_tokens=100)[0].numpy().tolist()))


































































































































































































































































MOF:
HBRDUEENOLIA:
ININWAN:
AUROLAOLI: od the O:
ARCut-
Fo lelle h aver t rstwathit bellly poenly ll
